<a href="https://colab.research.google.com/github/HalooMo/rep/blob/main/Computer_Visilon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#############################################################################################################################################################
#
#
#||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||
#||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||
#||||||||||||||||||||||||                                                                              ||||||||||||||||||||||||||||||||||||||||||||||||||||||
#||||||||||||||||||||||||                                     DOWNLOAD ENGLISH-SPANISH                 ||||||||||||||||||||||||||||||||||||||||||||||||||||||
#||||||||||||||||||||||||                                                                              ||||||||||||||||||||||||||||||||||||||||||||||||||||||
#||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||
#||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||



!wget http://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip
!unzip -q spa-eng.zip

--2024-07-14 05:15:42--  http://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.132.207, 74.125.201.207, 74.125.202.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.132.207|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2638744 (2.5M) [application/zip]
Saving to: ‘spa-eng.zip’

spa-eng.zip         100%[===================>]   2.52M  --.-KB/s    in 0.01s   

2024-07-14 05:15:43 (251 MB/s) - ‘spa-eng.zip’ saved [2638744/2638744]



In [ ]:
#############################################################################################################################################################
#
#
#||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||
#||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||
#||||||||||||||||||||||||                                                                              ||||||||||||||||||||||||||||||||||||||||||||||||||||||
#||||||||||||||||||||||||                                     Transformer Model                        ||||||||||||||||||||||||||||||||||||||||||||||||||||||
#||||||||||||||||||||||||                                                                              ||||||||||||||||||||||||||||||||||||||||||||||||||||||
#||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||
#||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||



import tensorflow as tf
import random
import tensorflow.keras as keras
import string
import numpy as np
import re


sequence_length = 20
max_tokens = 15000
embed_dim = 256
dense_dim = 128
num_heads = 4


file_path = "spa-eng/spa.txt"
pairs = []
file = open(file_path, "r")
for f in file.readlines():
  english, spanish = f.split("\t")
  spanish = "[start]" + spanish + "[end]"
  pairs.append((english, spanish))
random.shuffle(pairs)

pair_len = int(len(pairs)//2)
train = pairs[pair_len:]
val = pairs[:pair_len]




punct = string.punctuation + "¿"
punct.replace("[", "")
punct.replace("]", "")


def stand_art(str_ing):
  lower_case = tf.strings.lower(str_ing)
  return tf.strings.regex_replace(lower_case, f"[{re.escape(punct)}]", "")

english_vectorization = keras.layers.TextVectorization(
    max_tokens,
    output_mode = "int",
    output_sequence_length = sequence_length
)


spanish_vectorization = keras.layers.TextVectorization(
    max_tokens,
    output_mode = "int",
    output_sequence_length = sequence_length + 1,
    standardize = stand_art,
)

english = [i[0] for i in pairs]
spanish = [i[1] for i in pairs]
english_vectorization.adapt(english)
spanish_vectorization.adapt(spanish)


def format_data(eng,spa):
  spa = spanish_vectorization(spa)
  eng = english_vectorization(eng)
  return ({
      "english":eng,
      "spanish":spa[:, :-1],
      }, spa[:, 1:])

def make_ds(data):
  eng, spa = zip(*data)
  eng = list(eng)
  spa = list(spa)
  ds = tf.data.Dataset.from_tensor_slices((eng, spa))
  ds = ds.batch(32)
  ds = ds.map(format_data, num_parallel_calls=4)
  return ds.shuffle(2020).prefetch(16).cache()


train_ds = make_ds(train)
val_ds = make_ds(val)


for input, label in train_ds.take(1):
  print("input english shape  --- " + str(input["english"].shape))
  print("input spanishish shape  --- " + str(input["spanish"].shape))
  print("label shape  --- " + str(label.shape))




class TransformerEmbedding(keras.layers.Layer):
  def __init__(self, embed_dim, num_heads, dense_dim, **kwargs):
    super().__init__(**kwargs)
    self.embed_dim = embed_dim
    self.num_heads = num_heads
    self.dense_dim = dense_dim
    self.attention = keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
    self.dense_lay = keras.Sequential([keras.layers.Dense(dense_dim, activation="relu"),
                       keras.layers.Dense(embed_dim)])

    self.norm1 = keras.layers.LayerNormalization()
    self.norm2 = keras.layers.LayerNormalization()


  def call(self, inputs, mask=None):
    if mask is not None:
      mask = mask[:,tf.newaxis,:]
    x = self.attention(inputs, inputs, attention_mask=mask)
    x = self.norm1(inputs+x)
    d = self.dense_lay(x)
    return self.norm2(d+x)

  def get_config(self):
    conf = super().get_config()
    conf.update(
        {
            "embed_dim":self.embed_dim,
            "num_heads":self.num_heads,
            "dense_dim":self.dense_dim
        }
    )
    return conf


class PositionlEmbedding(keras.layers.Layer):
  def __init__(self, embed_dim, max_tokens, sequence_length, **kwargs):
    super().__init__(**kwargs)
    self.embed_dim = embed_dim
    self.max_tokens = max_tokens
    self.sequence_length = sequence_length
    self.embedding = keras.layers.Embedding(self.max_tokens, self.embed_dim)
    self.positional_embedding = keras.layers.Embedding(self.sequence_length, self.embed_dim)


  def call(self, inputs):
    lim = tf.shape(inputs)[-1]
    period = tf.range(start=0, limit=lim, delta=1)
    emb = self.embedding(inputs)
    pos_emb = self.positional_embedding(period)
    return emb + pos_emb

  def get_config(self):
    conf = super().get_config()
    conf.update(
        {
            "embed_dim":self.embed_dim,
            "sequence_length":self.sequence_length,
            "max_tokens":self.max_tokens
        }
    )
    return conf



class TransformerDecoder(keras.layers.Layer):
  def __init__(self, embed_dim, num_dense, num_heads, **kwargs):
    super().__init__(**kwargs)
    self.embed_dim = embed_dim
    self.num_dense = num_dense
    self.num_heads = num_heads
    self.norm1 = keras.layers.LayerNormalization()
    self.norm2 = keras.layers.LayerNormalization()
    self.norm2 = keras.layers.LayerNormalization()
    self.supports_masking = True
    self.multi_attention1 = keras.layers.MultiHeadAttention(
        key_dim=embed_dim,
        num_heads=num_heads
    )
    self.multi_attention2 = keras.layers.MultiHeadAttention(
        key_dim=embed_dim,
        num_heads=num_heads
    )
    self.denses = keras.Sequential([
        keras.layers.Dense(num_dense, "relu"),
        keras.layers.Dense(embed_dim)
        ])

    def get_config(self):
      conf = super().get_config()
      conf.update({
          "embed_dim": embed_dim,
          "num_heads": num_heads,
          "num_dense": num_dense,
      })
      return conf

    def custom_mask(self, inputs):
      length = tf.shape(inputs)
      batch_size, sequence_length = length[0], length[1]
      i = tf.range(sequence_length)[:, tf.newaxis]
      j = tf.range(sequence_length)
      mask = tf.cast(i>=j, dtype="int32")
      mask = mask.reshape(mask, (1, sequence_length, sequence_length))
      mult = tf.concat([
          tf.expand_dims(batch_size,-1),
          tf.constant([1,1],dtype = tf.int32)],
          axis=0
      )
      return tf.tile(mask,mult)


    def call(self, inputs, mask = None):
      custom_mask = self.custom_mask(inputs)
      if mask is not None:
        padding_mask = tf.cast(mask[:,tf.newaxis,:], dtype="int32")
        padding_mask = tf.minimum(padding_mask, custom_mask)
      x = self.multi_attention(
          query=inputs,
          value=inputs,
          key=inputs,
          attention_mask=custom_mask
      )
      n1 = self.norm1(inputs+x)
      x = self.multi_attention2(
          inputs,
          inputs,
          inputs,
          attention_mask = padding_mask
      )
      x = self.norm2(n1+x)
      denses = self.denses(x)
      return self.norm3(denses+x)


input_e = keras.Input(shape=(None,), dtype="int64", name="english")
x = PositionlEmbedding(embed_dim, max_tokens, sequence_length)(input_e)
outp = TransformerEmbedding(embed_dim, num_heads, dense_dim)(x)


input_s = keras.Input(shape=(None,), dtype="int64", name="spanish")
x = PositionlEmbedding(embed_dim, max_tokens, sequence_length)(input_s)
x = TransformerDecoder(embed_dim, dense_dim, num_heads)(x, outp)
x = keras.layers.Dropout(0.5)(x)
outp = keras.layers.Dense(max_tokens, "softmax")(x)


model = keras.Model([input_e, input_s], outp)





model.compile(
    "rmsprop",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)



model.fit(
    train_ds,
    epochs=5,
    validation_data=val_ds
)


vocab = spanish_vectorization.get_vokebulary()
un_vocab = dict(zip(range(len(vocab)), vocab))

def get_sequence(input_sequence):
  eng = english_vectorization([input_sequence])
  span = "[start]"
  for i in range(len(input_sequence)):
    spanish = spanish_vectorization([span])[:, :-1]
    rnn_predict = model([eng, span])
    text = np.argmax(rnn_predict[0, i, :])
    span += " " + un_vocab[text]
    if un_vocab[text] == "[end]":
      break
  return span + "end"







input english shape  --- (32, 20)
input spanishish shape  --- (32, 20)
label shape  --- (32, 20)
Epoch 1/5


ValueError: in user code:

    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1401, in train_function  *
        return step_function(self, iterator)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1384, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1373, in run_step  **
        outputs = model.train_step(data)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1154, in train_step
        self.optimizer.minimize(loss, self.trainable_variables, tape=tape)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/base_layer.py", line 2271, in trainable_variables
        return self.trainable_weights
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 3010, in trainable_weights
        trainable_variables += t

In [ ]:
import keras
import tensorflow as tf
import numpy as np
import pathlib
import os
import shutil


(data, labels), _ = keras.datasets.mnist.load_data()


input = keras.Input(shape=(28, 28, 1))
x = keras.layers.Conv2D(16, kernel_size=3, activation="relu")(input)
x = keras.layers.MaxPooling2D()(x)
x = keras.layers.Conv2D(32,kernel_size=3, activation="relu")(x)
x = keras.layers.MaxPooling2D()(x)
x = keras.layers.Conv2D(64,kernel_size=3, activation="relu")(x)
x = keras.layers.Flatten()(x)
output = keras.layers.Dense(10, "sigmoid")(x)

model = keras.Model(input, output)

model.compile("rmsprop",
              "sparse_categorical_crossentropy",
              ["accuracy"]
)
model.fit(
    data,
    labels,
    epochs=4
)






Epoch 1/4
1875/1875 [==============================] - 30s 16ms/step - loss: 0.2645 - accuracy: 0.9446
Epoch 2/4
1875/1875 [==============================] - 29s 16ms/step - loss: 0.0684 - accuracy: 0.9810
Epoch 3/4
1875/1875 [==============================] - 30s 16ms/step - loss: 0.0536 - accuracy: 0.9856
Epoch 4/4
1875/1875 [==============================] - 29s 15ms/step - loss: 0.0458 - accuracy: 0.9877


In [ ]:
from google.colab import files
files.upload()

Saving WIN_20240107_10_43_49_Pro.jpg to WIN_20240107_10_43_49_Pro.jpg
Saving WIN_20240528_17_47_35_Pro.jpg to WIN_20240528_17_47_35_Pro.jpg
Saving WIN_20240528_17_47_49_Pro.jpg to WIN_20240528_17_47_49_Pro.jpg


{'WIN_20240107_10_43_49_Pro.jpg': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x01\x00`\x00`\x00\x00\xff\xe10\xcaExif\x00\x00MM\x00*\x00\x00\x00\x08\x00\x04\x011\x00\x02\x00\x00\x00\x0b\x00\x00\x10J\x87i\x00\x04\x00\x00\x00\x01\x00\x00\x10V\x88%\x00\x04\x00\x00\x00\x01\x00\x00 \xa2\xea\x1c\x00\x07\x00\x00\x10\x0c\x00\x00\x00>\x00\x00\x00\x00\x1c\xea\x00\x00\x00\x10\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x0